## RNN 做文本生成
用word 级别来做
我们这里的文本预测就是，给了前面的单词以后，下一个单词是谁？
比如，hello from the other, 给出 side

英文预料：古登堡计划网站下载txt平文本：https://www.gutenberg.org/wiki/Category:Bookshelf)

第一步，先导入各种库

In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models.word2vec import Word2Vec

Using TensorFlow backend.
d:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


读入文本

In [2]:
raw_text = ''
for file in os.listdir("input/"):
    if file.endswith(".txt"):
        raw_text += open("input/"+file,encoding='utf-8', errors='ignore').read() + '\n\n'
# raw_text = open('../input/Winston_Churchil.txt').read()
raw_text = raw_text.lower() # 都变成小写
sentensor = nltk.data.load('tokenizers/punkt/english.pickle')   # 分句
sents = sentensor.tokenize(raw_text) # 分词
corpus = []
for sen in sents:
    corpus.append(nltk.word_tokenize(sen))

print(len(corpus))
print(corpus[:3])

77294
[['\ufeff', 'twenty', 'years', 'after', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.'], ['you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'http', ':', '//www.gutenberg.org/license', '.'], ['if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', ',', 'you', '’', 'll', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', '.']]


用word2vec 训练词向量

In [3]:
w2v_model=Word2Vec(corpus,size=128,window=5,min_count=5,workers=4)

In [4]:
w2v_model['office']

array([ -1.70100689e-01,   2.21460104e-01,  -5.34742922e-02,
        -3.39673340e-01,   1.77219138e-01,  -2.05686405e-01,
         1.34869337e-01,  -7.68376887e-02,  -1.71628296e-01,
         5.59415556e-02,   5.84180467e-02,   7.74969533e-02,
         8.75741690e-02,  -1.79418430e-01,   4.28613037e-01,
         8.92448053e-02,  -1.04237899e-01,  -2.80936301e-01,
         2.15246782e-01,   2.99152374e-01,  -1.17879227e-01,
        -4.94978800e-02,   9.56530049e-02,  -1.87774166e-01,
        -3.28286648e-01,   4.13469851e-01,   2.80880779e-01,
         4.50759262e-01,  -3.73906106e-01,  -1.05421439e-01,
        -3.49848598e-01,   1.53443813e-01,  -3.10900748e-01,
         2.04842001e-01,   7.11158633e-01,  -4.84331548e-01,
        -1.69602007e-01,  -2.75364816e-01,   9.72375646e-02,
         1.91466376e-01,   4.30587023e-01,   3.45847696e-01,
        -6.38228143e-03,   3.97485465e-01,   1.51778966e-01,
        -5.76866388e-01,  -7.60715008e-02,  -1.20294556e-01,
        -1.48702627e-02,

In [5]:
type(w2v_model['office'])

numpy.ndarray

In [6]:
w2v_model['office'].shape # 每个单词为128维向量

(128,)

处理我们的training data，把源数据变成一个长长的x，好让LSTM学会predict下一个单词：

In [7]:
raw_input=[item for sublist in corpus for item in sublist]
len(raw_input) # 一共这么多个单词

1674820

In [8]:
raw_input[12] # 比如看看第13个单词是啥

'anywhere'

In [9]:
# 由于word2vec 训练时忽略掉了次数小于5的单词，这些词没有词向量，所以过滤掉这些词
text_stream=[]
vocab = w2v_model.wv.vocab  # w2v_model.vocab
for word in raw_input:
    if word in vocab:
        text_stream.append(word)
len(text_stream) # 过滤掉之后剩下的单词数

1610628

### 构造训练测试集
将 text_stream 变成可以用来训练的x,y:
x 是前 seq_length个字母，y 是后一个字母

In [10]:
seq_length=10
x=[]
y=[]
for i in range(0,len(text_stream)-seq_length):
    given=text_stream[i:i+seq_length]
    predict=text_stream[i+seq_length]
    x.append( np.array([ w2v_model[word] for word in given]) ) #
    y.append( w2v_model[predict] )

In [11]:
print(x[10])
print(y[10])

[[ 0.0315582   0.24742642  0.29473412 ..., -0.10257404 -0.24917811
   0.1610685 ]
 [ 0.01192663  0.12200874  0.23699829 ...,  0.09416112 -0.04142332
   0.23037018]
 [ 0.06315383  0.59978247  0.65551829 ..., -0.1895072   0.32071814
  -0.44224665]
 ..., 
 [-0.86598939  1.40320039  0.72081333 ..., -0.00419424  0.12443067
   0.38223574]
 [ 0.59775227 -0.67513627 -0.44468141 ...,  0.62417305 -0.38476896
   0.84131485]
 [ 0.10815871  0.05645576 -0.19100019 ...,  0.10955054 -0.24792205
   0.42896098]]
[  5.28161407e-01   3.88861239e-01  -6.23571336e-01   1.45045757e+00
   2.54875493e+00  -4.13271219e-01   2.00851753e-01   2.93023251e-02
  -9.66639742e-02  -9.68550205e-01   1.98360360e+00  -1.40464497e+00
   2.74905294e-01  -2.74265409e-01  -1.01635468e+00  -3.77958789e-02
  -8.65953445e-01  -1.06454480e+00   1.03226149e+00   4.96265352e-01
  -6.67271018e-01   1.20532477e+00  -8.60495925e-01  -3.98858190e-01
   1.30271006e+00   9.68776122e-02   4.86399949e-01   2.51258820e-01
  -9.58744109e-01

In [12]:
print(type(y[10])) # y 中每个元素都是numpy array
print(type(x[10][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [13]:
print(len(x)) # 一共样本数
print(len(y))
print(len(x[12])) # 每个样本的单词向量数
print(len(x[12][0])) # 每个单词向量的维数
print(len(y[12])) 

1610618
1610618
10
128
128


转换为LSTM需要的数据格式 [样本数，时间步伐，特征]

In [14]:
x = np.reshape(x, (-1, seq_length, 128))
y = np.reshape(y, (-1,128))

In [15]:
type(x)

numpy.ndarray

### 模型构造
LSTM模型构造

In [16]:
model=Sequential()
model.add(LSTM(28, dropout_W=0.2, dropout_U=0.2, input_shape=(seq_length, 128))) #28是神经元个数,用256效果更好
model.add(Dropout(0.2))
model.add(Dense(128,activation='sigmoid'))
model.compile(loss='mse',optimizer='adam')

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(28, input_shape=(10, 128), dropout=0.2, recurrent_dropout=0.2)`
  


In [17]:
model.fit(x,y,nb_epoch=50,batch_size=2048)

d:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
1610618/1610618 [==============================] - 524s - loss: 0.6608   
Epoch 2/50
1610618/1610618 [==============================] - 173s - loss: 0.6364   
Epoch 3/50
1610618/1610618 [==============================] - 177s - loss: 0.6302   
Epoch 4/50
1610618/1610618 [==============================] - 178s - loss: 0.6263   
Epoch 5/50
1610618/1610618 [==============================] - 181s - loss: 0.6235   
Epoch 6/50
1610618/1610618 [==============================] - 178s - loss: 0.6208   
Epoch 7/50
1610618/1610618 [==============================] - 183s - loss: 0.6188   
Epoch 8/50
1610618/1610618 [==============================] - 179s - loss: 0.6170   
Epoch 9/50
1610618/1610618 [==============================] - 176s - loss: 0.6156   
Epoch 10/50
1610618/1610618 [==============================] - 176s - loss: 0.6146   
Epoch 11/50
1610618/1610618 [==============================] - 175s - loss: 0.6136   
Epoch 12/50
1610618/1610618 [==============================] - 

### 效果检验

In [18]:
def predict_next(input_array):
    x=np.reshape(input_array,(-1,seq_length,128))
    y=model.predict(x)
    return y

def string_to_index(raw_input):
    raw_input=raw_input.lower()
    input_stream=nltk.word_tokenize(raw_input)
    res=[]
    for word in input_stream[(len(input_stream)-seq_length):]:
        res.append(w2v_model[word])
    return res

def y_to_word(y):
    word=w2v_model.most_similar(positive=y,topn=1)
    return word

In [19]:
def generate_article(init,rounds=30):
    in_string=init.lower()
    for i in range(rounds):
        n=y_to_word(predict_next(string_to_index(in_string)))
        in_string+=' '+n[0][0]
    return in_string

In [20]:
init = 'Language Models allow us to measure how likely a sentence is, which is an important for Machine'
article = generate_article(init)
print(article)

language models allow us to measure how likely a sentence is, which is an important for machine due due terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible terrible
